In [ ]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

In [2]:
from lavis.datasets.builders import load_dataset
msvd_dataset = load_dataset("msvd_qa")

print(msvd_dataset.keys())
# dict_keys(['train', 'val', 'test'])

print(len(msvd_dataset["train"]))
# 566747

print(msvd_dataset["train"][0])

100%|██████████| 5278177/5278177 [00:00<00:00, 42723309.18it/s]


100%|██████████| 1094822/1094822 [00:00<00:00, 6273374.86it/s]


100%|██████████| 2245356/2245356 [00:00<00:00, 29622785.70it/s]


100%|██████████| 37623/37623 [00:00<00:00, 5271498.23it/s]

dict_keys(['train', 'val', 'test'])
30933
{'video': '/home/ommos92/.cache/lavis/msvd/videos/-4wsuPCjDBc_5_15.avi', 'text_input': 'what is chewing on a nut?', 'answers': 35, 'question_id': '0000000', 'instance_id': '0'}


In [3]:
#Write a function to convert the .avi into a set of images. Then we can randomly 
# sample from this set of images.
import cv2
from PIL import Image
import random

def convert_avi(data):
    # Open the video file
    cap = cv2.VideoCapture(data['video'])

    images = []
    while(cap.isOpened()):
        # Read the video frame by frame
        ret, frame = cap.read()
        if ret:
            # Convert the frame to RGB (OpenCV uses BGR by default)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Convert the frame to a PIL Image and append to the list
            images.append(Image.fromarray(frame))
        else:
            break

    # Close the video file
    cap.release()

    # Return the list of PIL Images
    return images

def random_sample(images, num_samples):
    #Randomly select 'num_samples' images from the list
    sampled_images = random.sample(images, num_samples)
    return sampled_images


images = convert_avi(msvd_dataset["train"][0])
print(len(images))


300


In [ ]:

# setup device to use
device = torch.device("cuda:1") if torch.cuda.is_available() else "cpu"

In [ ]:
#Load the model
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_opt", model_type="pretrain_opt6.7b", is_eval=True, device=device
)
